In [1]:
import numpy as np
import pandas as pd 
import os
from sklearn.model_selection import train_test_split
import PIL
import tensorflow as tf

In [2]:
X = []
y = []

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if len(dirname) <= 60:
            image_directory = dirname + '/' + filename
            image = PIL.Image.open(image_directory).resize((96, 96)).convert('L')
            
            # display(image)
            image_matrix = np.array(image).reshape([96,96,1])/255
            
            X.append(image_matrix)
            y.append(0) if 'n' in filename.lower()[:-3] else y.append(1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
print(f"Size of training set: {len(X_train)}")
print(f"Size of test set: {len(X_test)}")

Size of training set: 189
Size of test set: 64


In [6]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
#y_test = tf.keras.utils.to_categorical(y_test, 2)
y_test = np.array(y_test)

In [7]:
def generate_model():
    model = tf.keras.Sequential([
        # first layer (convolutional)
        tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        
        # second layer (convolutional)
        tf.keras.layers.Conv2D(64,kernel_size=(3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(0.2),
        
        # classifier
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

In [8]:
model = generate_model()
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy()])

2022-03-05 15:57:07.022531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 15:57:07.136603: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 15:57:07.137321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 15:57:07.138461: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
model.fit(X_train, y_train, epochs=16, batch_size=4, verbose=1)

2022-03-05 15:57:09.389799: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/16


2022-03-05 15:57:10.694772: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


48/48 [==============================] - 7s 6ms/step - loss: 0.6236 - binary_accuracy: 0.6455
Epoch 2/16
48/48 [==============================] - 0s 6ms/step - loss: 0.5423 - binary_accuracy: 0.7249
Epoch 3/16
48/48 [==============================] - 0s 5ms/step - loss: 0.4624 - binary_accuracy: 0.8095
Epoch 4/16
48/48 [==============================] - 0s 5ms/step - loss: 0.4973 - binary_accuracy: 0.7778
Epoch 5/16
48/48 [==============================] - 0s 6ms/step - loss: 0.3551 - binary_accuracy: 0.8413
Epoch 6/16
48/48 [==============================] - 0s 5ms/step - loss: 0.3219 - binary_accuracy: 0.8783
Epoch 7/16
48/48 [==============================] - 0s 5ms/step - loss: 0.3156 - binary_accuracy: 0.8466
Epoch 8/16
48/48 [==============================] - 0s 6ms/step - loss: 0.2095 - binary_accuracy: 0.9153
Epoch 9/16
48/48 [==============================] - 0s 5ms/step - loss: 0.1829 - binary_accuracy: 0.9312
Epoch 10/16
48/48 [==============================] - 0s 6ms/step -

In [10]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

Evaluate on test data
64/64 [==============================] - 0s 2ms/step - loss: 1.1149 - binary_accuracy: 0.7812
test loss, test acc: [1.1148594617843628, 0.78125]
